In [4]:
from transformers import BitsAndBytesConfig

from datasets import load_dataset

# Data

In [2]:
system_message = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's question."

In [5]:
dataset_id = "euclid-multimodal/Geoperception"
dataset = load_dataset(dataset_id)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/93.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11657 [00:00<?, ? examples/s]

In [6]:
dataset_linecomparison = dataset['train'].filter(lambda x: x['predicate'] == 'LineComparison')
train_test = dataset_linecomparison.train_test_split(test_size=0.5,seed=42)

Filter:   0%|          | 0/11657 [00:00<?, ? examples/s]

In [8]:
train_test

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'predicate', 'image'],
        num_rows: 697
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'predicate', 'image'],
        num_rows: 697
    })
})

In [ ]:
train_tesxt

In [10]:
def format_data(sample):
    return [
        {"role":"system", "content":[{"type":"text","text":system_message}]},
        {"role":"user", "content":[{"type":"image","image":sample['image']}]},
        {"role":"assistant","content":[{"type":"text","text":sample['answer']}]}
    ]

In [11]:
train_dataset = [format_data(x) for x in train_test['train']]
train_dataset

[[{'role': 'system',
   'content': [{'type': 'text',
     'text': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's question."}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=305x214>}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': 'DB'}]}],
 [{'role': 'system',
   'content': [{'type': 'text',
     'text': "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's question."}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=555x450>}]},
  {'role': 'assistant', 'content': [{'type': 'text', 'text': 'UT'}]}],
 [{'role': 'system',
   'content': [{'type': 'text',
     'text': "A chat between a curious user and an artificial intelligence assistant. T

# Model